In [1]:
import pandas as pd
import numpy as np
import urllib.request, urllib.parse, urllib.error, io

# Static NBA API 
import random
from nba_api.stats.static import players,teams
from nba_api.stats.endpoints import playercareerstats,playergamelog
from scipy import ndimage
# NBA API Endpoint

from nba_api.stats.endpoints import teamdashptpass# For team passing stats
from nba_api.stats.endpoints import playerdashptpass # For player passing stats
from nba_api.stats.endpoints import ShotChartDetail # For player shot location and frequency
from nba_api.stats.endpoints import commonplayerinfo # For Common Player Information
#from nba_api.stats.endpoints import playergamelogs

#from nba_api.stats.endpoints import teamgamelogs
# Plotting Libraries

from matplotlib.patches import Circle, Rectangle, Arc
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib.colors import LinearSegmentedColormap

from matplotlib.patches import Circle, Rectangle, Arc

from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib.colors import LinearSegmentedColormap

import matplotlib.pyplot as plt
import functools
import typing
import asyncio
import os
'''
seasons=[]
for year in range(1997,2024):
    season = str(year-1) +'-'+str(year)[-2:]
    seasons.append(season)
league_id = '00'
for season in seasons:
    #print(season)
    shot_chart = ShotChartDetail(
                      player_id ='0',
                      
                      team_id='1610612737',
                      season_nullable= season,
                      season_type_all_star='Regular Season', 
                      context_measure_simple= 'FGA',
                                    league_id = league_id).get_data_frames()
    carry = ''
    avg = shot_chart[1]
    year = int(season[0:4])+1
    path = 'team/'+str(year)+carry+'/avg.csv'
    #print(path)
    avg.to_csv(path,index = False)
    #print(avg)
    shot_chart = ShotChartDetail(
                      player_id ='0',
                      
                      team_id='1610612737',
                      season_nullable= season,
                      season_type_all_star='Playoffs', 
                      context_measure_simple= 'FGA',
                                    league_id = league_id).get_data_frames()
    carry = 'ps'
    avg = shot_chart[1]
    path = 'team/'+str(year)+carry+'/avg.csv'
    avg.to_csv(path,index = False)
    print('Season '+season +' done') 
'''
def team_shotmaps_orig():
    nba_teams = teams.get_teams()
    team_ids =[]
    for org in nba_teams:
        team_ids.append(org['id'])


    seasons = []
    path = 'team_orig/'
    league_id='00'
    for year in range(1996,2023):
        year_path = path+str(year)+'/'
        isExist = os.path.exists(year_path)
        if not isExist:

       # Create a new directory because it does not exist
            os.makedirs(year_path)
            print('Making Folder ' +year_path)
        year_path_ps = path+str(year)+'ps'+'/'

        isExist = os.path.exists(year_path_ps)
        if not isExist:

       # Create a new directory because it does not exist
            os.makedirs(year_path_ps)
            print('Making Folder ' +year_path_ps)
        season =  str(year)+'-'+str(year+1)[2:4]
        print(str(year) + ' Started')
        for team_id in team_ids:

            shot_chart = ShotChartDetail(
                                  player_id ='0',

                                  team_id=team_id,
                                  season_nullable= season,
                                  season_type_all_star='Regular Season', 
                                  context_measure_simple= 'FGA',
            league_id = league_id).get_data_frames()
            team_shots = shot_chart[0]
            team_shots.to_csv(year_path+str(team_id)+'.csv',index = False)
            
            shot_chart = ShotChartDetail(
                                  player_id ='0',

                                  team_id=team_id,
                                  season_nullable= season,
                                  season_type_all_star='Playoffs', 
                                  context_measure_simple= 'FGA',
            league_id = league_id).get_data_frames()
            team_shots = shot_chart[0]
            team_shots.to_csv(year_path_ps+str(team_id)+'.csv',index = False)
        print(str(year) + ' done')
#team_shotmaps_orig()

In [2]:
frames = []
for i in range(2014,2017):
    df = pd.read_csv('team_orig/'+str(i)+'/1610612739.csv')
    frames.append(df)

cle= pd.concat(frames)

print(len(cle[cle.PLAYER_ID==202681]))

3531


In [7]:
import os
def fix_teamdata(ps=''):
    for year in range(1997,2024):
        
        year_orig = year -1
        year = str(year)
        year_orig = str(year_orig)
        path= 'team/'+year+ps+'/'
        files = [file for file in os.listdir(path) if '.csv' in file and 'vs' not in file and 'avg' not in file]
        for file in files:
            full_path = path+file
            df = pd.read_csv(full_path)
            
            #print(df.columns)
            #print(len(df.columns))

            if  'Unnamed: 0' in df.columns:
                df.drop(columns = 'Unnamed: 0',inplace = True)
            #print(len(df))
            df2 = pd.read_csv('team_orig/'+year_orig+ps+'/'+file)
            #print(df.head(30))
            df['SHOT_ID'] = df['SHOT_ID'].astype(str)
            df2['SHOT_ID'] = df2['GAME_ID'].astype(str)+df2['GAME_EVENT_ID'].astype(str)
            small_frame = df2[['PLAYER_ID','PLAYER_NAME','SHOT_ID']]
            df = df.drop(columns =['PLAYER_ID','PLAYER_NAME'])
            final = small_frame.merge(df,on='SHOT_ID')
            #print(full_path)
            #print(len(df))
            #print(len(df2))
            #print(len(final))
            #print(df.head(30))
            #print(df2.head(30))
            #print(final.head(30))
            #print(len(final.columns))
            
            #break
            final.to_csv(full_path,index = False)
        print(year + ' year done')
fix_teamdata()

fix_teamdata(ps='ps')
        


1997 year done
1998 year done
1999 year done
2000 year done
2001 year done
2002 year done
2003 year done
2004 year done
2005 year done
2006 year done
2007 year done
2008 year done
2009 year done
2010 year done
2011 year done
2012 year done
2013 year done
2014 year done
2015 year done
2016 year done
2017 year done
2018 year done
2019 year done
2020 year done
2021 year done
2022 year done
2023 year done
1997 year done
1998 year done
1999 year done
2000 year done
2001 year done
2002 year done
2003 year done
2004 year done
2005 year done
2006 year done
2007 year done
2008 year done
2009 year done
2010 year done
2011 year done
2012 year done
2013 year done
2014 year done
2015 year done
2016 year done
2017 year done
2018 year done
2019 year done
2020 year done
2021 year done
2022 year done
2023 year done
